<a href="https://colab.research.google.com/github/Birang1/opensource.guide/blob/main/%08Colab_5%EC%A3%BC%EC%B0%A8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
# 라이브러리 설치
!pip install yfinance pandas-datareader finance-datareader

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [16]:
# 라이브러리 가져오기 스니펫 : 선생님도 이걸 외워서 쓰지는 못하고 어디 붙혀놨던가 사용법을 보고 쓰는것임.
from pandas_datareader import data as pdr

import yfinance as yf
yf.pdr_override()

import numpy as np
import pandas as pd

import FinanceDataReader as fdr

df = fdr.DataReader('005930','2018')

df.head()

,Open,High,Low,Close,Volume,Change
Date,,,,,,
2018-01-02,51380,51400,50780,51020,169485,0.001177
2018-01-03,52540,52560,51420,51620,200270,0.011760
2018-01-04,52120,52180,50640,51080,233909,-0.010461
2018-01-05,51300,52120,51200,52120,189623,0.020360
2018-01-08,52400,52520,51500,52020,167673,-0.001919


In [ ]:
# 첫번째 백 테스팅 : 변동성 돌파 전략
# 변동성 돌파 전략이란?
# 주식이 오르는것 같다면 바로 사고 내일 바로 팔자!
# 오르는것 같다 의미? : (전일 최고 - 최저) * 0.5 + 오늘 주식 시작 가격 (시가) 보다 실제 종목 가격이 높아지면 사고 다음날 바로 팔자!

# 사야하는 날인가? 를 구분해보기

df = fdr.DataReader('005930','2018')

df.head()

k = 0.4

# 언제 사야하는가? colmn 추가 해주기, df['High or Low'].shift(1) 는 어제의 가격 이라고 생각하면 됨.
# buy_at : 사는 시점
df['buy_at'] = (df['High'].shift(1) - df['Low'].shift(1))*k + df['Open']

# 조건 해당 날의 High(최고가)가 buy_at(오르는 경향이 있는 가격)보다 높으면 살 수 있기 때문에 산다.
# is_buy : 사는 행위

df['is_buy'] = np.where(df['High'] > df['buy_at'], 'buy', '')

# 언제 파냐면 다음날의 open 보다 
df['sell_at'] = df['Open'].shift(-1)

# buy가 일어났던 순간들을 모아 수익률을 계산 하면 됨.


# is_buy가 buy인 것들만 모으기

df = df[df['is_buy'] == 'buy']

df['return'] = df['sell_at'] / df['buy_at']

df[['return']].cumprod().iloc[-2,-1] -1 
# cumprod()는 그 행의 누적의 곱임 : 산 사건들이 모두 발생해서 나타난 현상이 수익률 이기 때문에 이 함수를 활용
# 맨 마지막 행은 보통 NaN이라서 -2로 ㅐ함. / -1은 수익률 / # cumprod 할 때는 ()를 꼭 붙혀줘야함.




2.2903090629933747

In [ ]:
# 최적의 k 구하기

def get_return(code,k):

  df = fdr.DataReader(code,'2018')

  df['buy_at'] = (df['High'].shift(1) - df['Low'].shift(1))*k + df['Open']

  df['is_buy'] = np.where(df['High'] > df['buy_at'], 'buy', '')

  df['sell_at'] = df['Open'].shift(-1)

  df = df[df['is_buy'] == 'buy']

  df['return'] = df['sell_at'] / df['buy_at']

  return df[['return']].cumprod().iloc[-2,-1] -1 


In [ ]:
# 새로운 함수 등장 np.arange(x,y,n) x부터 y까지 n씩 상승합니다.
np.arange(0.4,0.6,0.01)

df = pd.DataFrame() # 빈 dataframe을 하나 만들고

for k in np.arange(0.4,0.6,0.01):
  doc = {
      'k' : k, # 쉼표 꼭 붙혀줘야 하고, =이 아니라 : 이다.
      'return' : get_return('005930',k)
  }
  df = df.append(doc, ignore_index = True) # df = df.append로 꼭 해줘야 함. 그냥 df.append 하면 안됨. # ,ignore_index = True를 꼭 붙혀줘야 한다.

df.sort_values(by = 'return', ascending = False)

# 코드도 자동화해서 종목별로 최적의 k를 찾을 수 있음. 
# 이것을 다루지는 않을것이지만 비슷한 방법을 이따가 적용할 것임.


<ipython-input-5-6edc1a8187ef>:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(doc, ignore_index = True) # df = df.append로 꼭 해줘야 함. 그냥 df.append 하면 안됨. # ,ignore_index = True를 꼭 붙혀줘야 한다.
<ipython-input-5-6edc1a8187ef>:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(doc, ignore_index = True) # df = df.append로 꼭 해줘야 함. 그냥 df.append 하면 안됨. # ,ignore_index = True를 꼭 붙혀줘야 한다.
<ipython-input-5-6edc1a8187ef>:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(doc, ignore_index = True) # df = df.append로 꼭 해줘야 함. 그냥 df.append 하면 안됨. # ,ignore_index = True를 꼭 붙혀줘야 한다.
<ipython-input-5-6edc1a8187ef>:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas i

,k,return
0,0.40,2.290309
1,0.41,2.118432
2,0.42,2.010280
3,0.43,1.941210
10,0.50,1.847459
4,0.44,1.768791
11,0.51,1.613797
5,0.45,1.596550
14,0.54,1.566525
6,0.46,1.526540


In [17]:
# 두번째 백 테스팅 :  월요일에 장이 열리면 사서 금요일에 파는걸 해보기
# 가장 적합한 종목은 무엇일지도 구해보기.

df = fdr.DataReader('005930','2018')


# Date column을 불러와야 하는데 되지 않음. 
# 날짜가 문자열로 인식돼서 그렇다. 
# 따라서, index를 reset 해줘야 한다.
# df[['Date']]

df = df.reset_index()

# 이렇게 하면 행을 구분짓는 날짜 (Date) 가 column의 하나로 들어가서 Date와 Open을 따로 부를 수 있다.

# Date와 Open이 필요함.

df = df[['Date','Open']]

# 요일 붙히기

# 먼저, Date를 날짜 형식으로 바꿔주기. (pd.to_datetime()이라는 날짜 형식으로 바꿔주는 함수 이용.)

df['Date'] = pd.to_datetime(df['Date'])

# text적으로 티는 나지 않음.
# df.dtypes 를 쳐보면 Date가 datetime으로 바뀐것을 확인 할 수 있다.

df.dtypes

# .dt.day_name()을 붙혀주면 Date column이 요일로 바뀌어진 Day라는 column이 추가 된다.
df['Day'] = pd.to_datetime(df['Date']).dt.day_name()

# 월요일하고 금요일만 먼저 빼보기

cond = (df['Day'] == 'Monday') | (df['Day'] == 'Friday')

# 주의 : 또는 은 | 이다. , 조건은 괄호 안에 하나씩 넣어줘야 한다.

df = df[cond]

# 월요일에 사서 금요일에 팔거기 때문에, 월요일에 시작해서 금요일에 끝나는 dataframe이 만들어져야한다.

# 첫번째가 금요일인것을 지우는 코드
if df.iloc[0,2] == 'Friday': # 첫번째 행의 3번째 column이 금요일 이라면
  df = df.drop(index = df.index[0]) #  첫번째 행 (index가 0인 행)을 지워주세요ㅣ

# 마지막이 월요일인것을 지우는 코드
if df.iloc[-1,2] == 'Monday': # 마지막 행의 3번째 column이 월요일 이라면
  df = df.drop(index = df.index[-1]) #  마지막 행 (index가 맨 마지막으로 표시된 행)을 지워주세요ㅣ

# 휴일을 고려한 코드 : 월요일에 샀는데, 금요일이 공휴일이여서 팔 수 없는 경우.
# 위 같은 경우라면 금요일이 공휴일인 주의 월요일이 들어가 있는 행을 지워줘야한다

# Day가 월요일이고, 한칸 위로 당긴 것을 옆으로 붙혔을때도 Monday인 경우
# 즉, 월요일이고 그주 금요일이 휴일이여서 다음주 월요일에 사는것으로 표현 되어 있는 경우 = cond
# 코드 풀이 : 월요일이 나왔는데 (그주 금요일이 휴일이라), 다음것이 바로 또 월요일 일때,
# shift(-1) : 다음주것을 당겨온다.
cond = (df['Day'] == 'Monday') & (df['Day'].shift(-1) == 'Monday')

# condition이 있는 모든 index를 drop해 달라!
df = df.drop(index = df[cond].index)

# 월요일이 휴일이였다면, 
# 그 주 월요일에 사라는 명령은 없고 금요일에 팔라는 명령만 있을것이니,
# 전주 월, 금 과 겹쳐 월, 금 , 금 으로 나올것이다.
# 따라서 월요일이 휴일인 주의 행을 빼줘야한다.
# 코드 풀이 : 금요일이 나왔는데 (그주 월요일이 휴일이라), 전것이 바로 또 금요일 일때,
# shift(1) : 전 주것을 당겨온다.
cond = (df['Day'] == 'Friday') & (df['Day'].shift(1) == 'Friday')
df = df.drop(index = df[cond].index)

# 이제 월금 월금 만 남음.

# 수익률 구하기

# monday - friday 해주기 위해서
# monday 옆에 그주 friday open 값을 배치 해주기
df['Open_fri'] = df['Open'].shift(-1)

# day가 monday인것만 남겨두기

df = df[df['Day'] == 'Monday']

# Monday_Open과 Friday_Open만 남겨두기

df = df[['Open','Open_fri']]

# column이름 바꾸기

df.columns = ['buy_at','sell_at']

# 수익률 column 추가
df['return'] = df['sell_at'] / df['buy_at']

# 수익률 column에서
# df = df[['return']]

# 수익률 구하기 위해 모두 곱하기

# df.cumprod() # 만약 이 과정에서 inf가 나온다면, 0인 값이 있어   n/0꼴이 되어 무한대 값을 가져서 그렇다.

# 분모와 분자가 모두 0이 아닌 경우를 condition으로 걸어서 cumprod 시키면 됨.
cond = (df['sell_at'] !=0) & (df['buy_at'] !=0)

df = df[cond]

df = df[['return']].cumprod()

df

,return
4,0.920611
9,0.950417
14,0.986761
19,0.951685
24,0.909524
...,...
1279,1.314304
1284,1.348721
1289,1.378884
1294,1.374575


In [18]:
# 위의것 함수로 자동화

def get_return_mf (code):
  df = fdr.DataReader(code,'2018')

  df = df.reset_index()

  df = df[['Date','Open']]

  df['Date'] = pd.to_datetime(df['Date'])

  df.dtypes

  df['Day'] = pd.to_datetime(df['Date']).dt.day_name()

  cond = (df['Day'] == 'Monday') | (df['Day'] == 'Friday')

  df = df[cond]

  if df.iloc[0,2] == 'Friday': 
    df = df.drop(index = df.index[0])


  if df.iloc[-1,2] == 'Monday': 
    df = df.drop(index = df.index[-1])

  cond = (df['Day'] == 'Monday') & (df['Day'].shift(-1) == 'Monday')

  df = df.drop(index = df[cond].index)

  cond = (df['Day'] == 'Friday') & (df['Day'].shift(1) == 'Friday')
  df = df.drop(index = df[cond].index)

  df['Open_fri'] = df['Open'].shift(-1)

  df = df[df['Day'] == 'Monday']

  df = df[['Open','Open_fri']]


  df.columns = ['buy_at','sell_at']

  df['return'] = df['sell_at'] / df['buy_at']

  cond = (df['sell_at'] !=0) & (df['buy_at'] !=0)

  df = df[cond]

  df = df[['return']].cumprod()

  return df.iloc[-1,-1] -1

get_return_mf('005930')

0.4127579551653535

In [ ]:
# Open Dart를 통해 월-금 전략이 가장 잘 맞는 종목 고르기

!pip install dart-fss

In [ ]:
# Open Dart를 통해 월-금 전략이 가장 잘 맞는 종목 고르기

# open dart site 들어가서
# 로그인하고
# 인증키 관리 > Open API 현황

# 종목가져오기 코드 스니펫

import dart_fss as dart_fss
import pandas as pd

api_key = '422485e457defdc63588e8a948b519cf41ea8793' #'API키 입력하기'
dart_fss.set_api_key(api_key=api_key)

all = dart_fss.api.filings.get_corp_code() # 모든 회사의 종목 코드를 가져오는데

df = pd.DataFrame(all) # 모든 회사의 종목 코드를 가져오는데

df_listed = df[df['stock_code'].notnull()] # 상장 기업만 가져오기


# Random으로 10개 sampling
for row in df_listed.sample(10)[['stock_code','corp_name']].itertuples():
  print(row)

In [ ]:
# Random으로 10개 sampling
# print(row[n]) 하면 n+1번째 column에 정보가 나올것임.
for row in df_listed.sample(10)[['stock_code','corp_name']].itertuples():
  print(row)

df=pd.DataFrame() # 빈 datarame 만들어주고
for row in df_listed.sample(10)[['stock_code','corp_name']].itertuples(): # random sampling 해서 row 들을 불러온다.
  try:
    doc = {
        'name' : row[2], # 이름추가 # ,쉼표 꼭 붙혀주기
        'return' : get_return_mf(row[1]) # 종목 코드 추가
    }
    df= df.append(doc,  ignore_index = True) # 쌓기
  except:
    print(f'error - {row[2]}') # 에러가 나면 종목 이름과 같이 표시해주기 # row에 중괄호 표시
df.sort_values(by = 'return', ascending = False)

# 위에서 sample(10)만 떼어내서 분석하면 전체 상장 종목에 대해서 분석 할 수 있음.
